In [1]:
# evaluate the inner class cluster 
import sys
import os
import json
VIS_METHOD='DVI'
sys.path.append('..')
epoch = 10
EPOCH_START = epoch
EPOCH_END = epoch
DEVICE="cuda:0"
CONTENT_PATH = '/home/yifan/dataset/clean/pairflip/cifar10/0'
from singleVis.data import NormalDataProvider
from trustVis.skeleton_generator import CenterSkeletonGenerator
sys.path.append(CONTENT_PATH)
with open(os.path.join(CONTENT_PATH, "config.json"), "r") as f:
    config = json.load(f)
config = config[VIS_METHOD]

# record output information
# now = time.strftime("%Y-%m-%d-%H_%M_%S", time.localtime(time.time())) 
# sys.stdout = open(os.path.join(CONTENT_PATH, now+".txt"), "w")

SETTING = config["SETTING"]
CLASSES = config["CLASSES"]
TRAINING_PARAMETER = config["TRAINING"]
NET = TRAINING_PARAMETER["NET"]
import Model.model as subject_model
net = eval("subject_model.{}()".format(NET))
data_provider = NormalDataProvider(CONTENT_PATH, net, EPOCH_START, EPOCH_END, 1, device=DEVICE, epoch_name='Epoch',classes=CLASSES,verbose=1)
train_data = data_provider.train_representation(EPOCH_START)
train_data = train_data.reshape(train_data.shape[0], train_data.shape[1])
train_data_pred = data_provider.get_pred(EPOCH_START, train_data)
train_data_pred_Res = train_data_pred.argmax(axis=1)

/home/yifan/miniconda3/envs/deepdebugger/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Finish initialization...


100%|██████████| 250/250 [00:00<00:00, 321.04it/s]


In [2]:
from singleVis.projector import DVIProjector
from singleVis.SingleVisualizationModel import VisModel

import numpy as np

VISUALIZATION_PARAMETER = config["VISUALIZATION"]
ENCODER_DIMS = VISUALIZATION_PARAMETER["ENCODER_DIMS"]
DECODER_DIMS = VISUALIZATION_PARAMETER["DECODER_DIMS"]
model = VisModel(ENCODER_DIMS, DECODER_DIMS)
from sklearn.cluster import KMeans

def get_overlap(train_data, emb, pred):
    kmeans = KMeans(n_clusters=2) 
    ratio_l = []
    for i in range(10):
        train_data_pred_for_sin = np.where(pred == i)[0]
        emb_for_sin = emb[train_data_pred_for_sin]
        train_data_sin = train_data[train_data_pred_for_sin]
        kmeans.fit(train_data_sin)
        labels_h = kmeans.labels_

        kmeans.fit(emb_for_sin)
        labels_l = kmeans.labels_

        mismatch_count = np.sum(labels_h != labels_l)
        # Computing the final ratio
        ratio = max(mismatch_count, len(labels_l) - mismatch_count) / len(labels_l)
        ratio_l.append(ratio)
        print("class{}, overlap rate is:{}".format(i, ratio))
    ave = sum(ratio_l) / len(ratio_l)
    print("avrage:{}".format(ave))
    return ave


In [26]:
VIS_MODEL_NAME = 'proxy' ### saved_as VIS_MODEL_NAME.pth

# Define Projector
projector = DVIProjector(vis_model=model, content_path=CONTENT_PATH, vis_model_name=VIS_MODEL_NAME, device=DEVICE)
emb = projector.batch_project(EPOCH_START, train_data)
av = get_overlap(train_data, emb)

Successfully load the DVI visualization model for iteration 150
class0, overlap rate is:0.7770445910817837
class1, overlap rate is:0.9264
class2, overlap rate is:0.5296940611877624
class3, overlap rate is:0.9126524085548671
class4, overlap rate is:0.8998200359928015
class5, overlap rate is:0.8866
class6, overlap rate is:0.9237542525515309
class7, overlap rate is:0.8309661932386477
class8, overlap rate is:0.9453781512605042
class9, overlap rate is:0.9448
avrage:0.8577109693867897


In [5]:
# VIS_MODEL_NAME =  'base'
# VIS_MODEL_NAME =  'trustbase_gen_border'
# epoch = 50
# Define Projector
projector = DVIProjector(vis_model=model, content_path=CONTENT_PATH, vis_model_name=VIS_MODEL_NAME, device=DEVICE)
emb = projector.batch_project(epoch, train_data)
av = get_overlap(train_data, emb, train_data_pred_Res)

Successfully load the DVI visualization model for iteration 50
class0, overlap rate is:0.5758638835292054
class1, overlap rate is:0.6123448726322664
class2, overlap rate is:0.5605786618444847
class3, overlap rate is:0.6227720696162717
class4, overlap rate is:0.6525057237344187
class5, overlap rate is:0.6062317429406037
class6, overlap rate is:0.5460873739798368
class7, overlap rate is:0.6000375023438965
class8, overlap rate is:0.6286089238845144
class9, overlap rate is:0.5175363558597091
avrage:0.5922567110365208


In [7]:
# VIS_MODEL_NAME =  'trustbase_gen_border'
VIS_MODEL_NAME = 'base'
epoch= 10
testing_data = data_provider.test_representation(epoch)
testing_data = testing_data.reshape(testing_data.shape[0], testing_data.shape[1])
testing_pred = data_provider.get_pred(epoch, testing_data).argmax(axis=1)
# Define Projector
projector = DVIProjector(vis_model=model, content_path=CONTENT_PATH, vis_model_name=VIS_MODEL_NAME, device=DEVICE)
emb = projector.batch_project(epoch, testing_data)
av = get_overlap(testing_data, emb,testing_pred)

100%|██████████| 50/50 [00:00<00:00, 14283.83it/s]


Successfully load the DVI visualization model for iteration 10
class0, overlap rate is:0.6690796277145812
class1, overlap rate is:0.9536307961504812


class2, overlap rate is:0.8342967244701349
class3, overlap rate is:0.9401960784313725
class4, overlap rate is:0.8113804004214963
class5, overlap rate is:0.9403973509933775
class6, overlap rate is:0.9650565262076053
class7, overlap rate is:0.7799227799227799
class8, overlap rate is:0.8992654774396642
class9, overlap rate is:0.8916256157635468
avrage:0.8684851377515039


In [18]:
# Define Projector
projector_base = DVIProjector(vis_model=model, content_path=CONTENT_PATH, vis_model_name='base', device=DEVICE)
ebm2 = projector_base.batch_project(epoch, train_data)
av = get_overlap(train_data, ebm2, train_data_pred_Res)

Successfully load the DVI visualization model for iteration 10
class0, overlap rate is:0.696401159815794


class1, overlap rate is:0.5243219339622641
class2, overlap rate is:0.5302579365079365
class3, overlap rate is:0.6207007393121183
class4, overlap rate is:0.6802177241027385
class5, overlap rate is:0.6419190076614374
class6, overlap rate is:0.5047185709470846
class7, overlap rate is:0.5354459327017315
class8, overlap rate is:0.5160554483242674
class9, overlap rate is:0.5785151667504609
avrage:0.5828553620085832
